# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
raw_data_file = "../output_data/raw_data.csv"
weather_data = pd.read_csv(raw_data_file)
weather_data.head()


,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Bredasdorp,ZA,1603033142,-34.53,20.04,69.80,52,59,13.87
1,Kiryat Gat,IL,1603033145,31.61,34.76,82.40,58,0,10.29
2,Albany,US,1603032871,42.60,-73.97,50.00,75,75,9.33
3,Punta Arenas,CL,1603033151,-53.15,-70.92,44.60,75,40,26.40
4,Ekhabi,RU,1603033155,53.51,142.97,35.28,93,96,23.58


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
weather_data = weather_data.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)

humidity = weather_data["Humidity"].astype(float)


In [6]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer1 = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer1)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:

#Narrow down cities to fit weather conditions outlines in readme
no_cloud = weather_data[weather_data["Cloudiness"] == 0]
above_70_F = no_cloud[no_cloud["Max Temperature"] > 70]
below_80_F = above_70_F[above_70_F["Max Temperature"] < 80]
slow_wind = below_80_F[below_80_F["Wind Speed"] < 10]
ideal_weather = slow_wind[slow_wind["Humidity"] < 55]

ideal_weather

# Set Index
indexed_ideal_weather = ideal_weather.reset_index()
del indexed_ideal_weather["index"]
indexed_ideal_weather

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Balad,IQ,1603033221,34.02,44.15,77.59,18,0,5.88
1,Vallenar,CL,1603034404,-28.57,-70.76,79.25,29,0,7.78
2,Tabas,IR,1603034671,33.60,56.92,71.60,8,0,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_ideal_weather["City"])):

    lat = indexed_ideal_weather.loc[city]["Latitude"]
    lng = indexed_ideal_weather.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "hotels",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_ideal_weather["Hotel Name"] = hotels
indexed_ideal_weather

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Nearest Hotel,Hotel Name
0,Balad,IQ,1603033221,34.02,44.15,77.59,18,0,5.88,رضاوي زنوبة,Ishaqi
1,Vallenar,CL,1603034404,-28.57,-70.76,79.25,29,0,7.78,Humacao Bed & Breakfast,Vallenar
2,Tabas,IR,1603034671,33.60,56.92,71.60,8,0,4.70,شرکت معماری مهراز شایگان آراد,Tabas


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_ideal_weather.iterrows()]
locations = indexed_ideal_weather[["Latitude", "Longitude"]]

In [32]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel Name: {hotel}" for hotel in hotels])
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))